**DTS - Complete Streets** *Right-of-Way Widths for Planned Street Improvements*



# 07 - Street Typology Overrides

**Author:** rmangan

---

**Purpose:**

After intial assingment of Complete Street Typologies, data was published to ArcGIS Online for review by DTS Staff. DTS Staff provided input on individual streets that were mis-assigned. Input was provided in the form of an ArcGIS Online point feature service, configured for editing by DTS staff with options to select a street type "override". Points were placed on the map on streets that needed adjustment.

**This script performs the following functions:**

1. Download AGOL comment layer via ArcGIS API for python
2. Spatial join comment point layer to modal composite layer.

**Manual processing was performed for the following functions**

3. Update complete street typology in modal composite based on complete street type in comment point layer. If DTS staff entered verbiage into the "Note" field, the comment is skipped and must be manually addressed/edited in ArcGIS Pro.

**Global Assumptions and Notes:**
1. All typology comments with values in "Note" field are skipped, comments must be manually addressed in ArcGIS Pro.


**Non-Standard Python Modules utilized:**
1. arcpy 2.7 - used for common data geo-processing
2. arcgis 1.83 - used for connecting to AGOL org and related items

In [ ]:
# import modules
import arcpy
import os
from arcgis.gis import GIS

In [ ]:
# set environment setttings
arcpy.env.workspace = "Z:\H\Honolulu_DTS\D3409300_RailActivation\GeoData\GDB\typology_comments.gdb"
arcpy.env.overwriteOutput = True

# login to AGOL
my_gis = GIS()

print("ArcGIS Online Org account")

pw = input("Enter AGOL password...")

gis = GIS("http://dithnl.maps.arcgis.com", "ray.mangan_ch2m", pw)
print("Logged in as " + str(gis.properties.user.username))


In [ ]:
# define variables
input_gdb_path = r"\\dc1vs01\GISProj\H\Honolulu_DTS\D3409300_RailActivation\GeoData\GDB\Input_Data.gdb"

modal_gdb = r"\\dc1vs01\GISProj\H\Honolulu_DTS\D3409300_RailActivation\GeoData\GDB\Modal\Modal Composite 5_3.gdb"

comment_gdb = r"\\dc1vs01\GISProj\H\Honolulu_DTS\D3409300_RailActivation\GeoData\GDB\typology_comments.gdb"


# Input Datasets

comment_REST = "https://services6.arcgis.com/2cZSk3EXXiOHcbOl/arcgis/rest/services/Complete_Streets_Comments/FeatureServer/0"

modal_composite = r"\\dc1vs01\GISProj\H\Honolulu_DTS\D3409300_RailActivation\GeoData\GDB\Modal\Modal Composite 5_3.gdb\modal_composite_05_3"

## 01 - Download AGOL feature service and spatial join to Modal Composite
---

Download the Street Type comments point layer and spatial join it to modal composite. Comment overrides are then manually reviewed and implemented by the project team.

**This section performs the following functions:**

1. Download AGOL comment point feature service

2. Spatial join point layer to modal composite line dataset


**Assumptions and Notes:**
1. none

In [ ]:
#copy cs layer to local gdb

comment_local = os.path.join(comment_gdb,"typology_comments_20210723")

print("Copying Comment layer to local gdb...")
arcpy.CopyFeatures_management(comment_REST, comment_local)
print("done")

#spatial join cs comments to rcl
sj_output = os.path.join(modal_gdb, "sj_output")

print("Running spatial join...")
arcpy.analysis.SpatialJoin(modal_composite, comment_local, sj_output, join_operation = "JOIN_ONE_TO_ONE", match_option = "CLOSEST", search_radius = 10)
print("done")
